In [1]:
import torch
import random
import numpy as np
import pandas as pd
import torch.utils.data as data
from tqdm.notebook import tqdm
import torch.nn as nn
import wandb

In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kubuseg. Use `wandb login --relogin` to force relogin


True

In [3]:
torch.cuda.set_device(0)
device = torch.device("cuda")
torch.cuda.get_device_name(0)

'NVIDIA RTX A3000 12GB Laptop GPU'

In [4]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Preprocess functions

In [5]:
def get_input_dataset(dataset_path):
    return np.load(dataset_path, allow_pickle=True)

In [6]:
def interpolate_missing_joints(movie):
    """
    Interpolate joint vals based on last valid value,
    number of invalid frames and current valid frame

    :param movie: has shape [Frame, Person, Axis, Joint]
    """
    last_val_joint, last_val_frame = (
        np.zeros_like(movie[0]),
        np.zeros_like(movie[0]),
    )
    for idx, frame in enumerate(movie):
        # Array that counts frames since last valid frame
        val_frame_diff = idx - last_val_frame
        interp_cond = (frame != 0) & (
            val_frame_diff > 1) & (last_val_joint != 0)
        # Iterate over (Person, Axis, Joint) tuples for interpolation
        for index in np.transpose(interp_cond.nonzero()):
            index = tuple(index)
            # index - 3 item tuple (Person, Axis, Joint)
            last_frame, num_frames = (
                int(last_val_frame[index]),
                int(val_frame_diff[index]),
            )
            delta = (frame[index] - last_val_joint[index]) / \
                val_frame_diff[index]
            for fr_num in range(1, num_frames):
                movie[(last_frame + fr_num,) + index] = (
                    last_val_joint[index] + delta * fr_num
                )
        last_val_joint[frame != 0] = frame[frame != 0]
        last_val_frame[frame != 0] = idx

In [7]:
def fill_border_joints(movie):
    """
    Fill first n invalid joint values frames with first valid

    :param movie: has shape [Person, Axis, Frame, Joint]

    """
    is_joint_valid = movie[0] != 0
    for idx, frame in enumerate(movie[1:], start=1):
        if is_joint_valid.all():
            break
        fill_cond = (frame != 0) & (is_joint_valid == False)
        for index in np.transpose(fill_cond.nonzero()):
            index = tuple(index)
            for fr_num in range(0, idx):
                movie[(fr_num,) + index] = frame[index]
        is_joint_valid[frame != 0] = True

In [8]:
def handle_missing_joints(movie):
    if (movie == 0).sum() == 0:
        return
    # Movie shape: (Frame[x], Person[2], Axis[3], Joint[25])
    interpolate_missing_joints(movie=movie)
    # Backward fill
    fill_border_joints(movie=movie)
    # Forward fill
    fill_border_joints(movie=movie[::-1])

In [9]:
def even_spaced_sampling(movie, out_frame_num):
    num_frames = movie.shape[2]
    if num_frames <= out_frame_num:
        return movie
    indexes = np.linspace(
        start=0, stop=num_frames, num=out_frame_num, endpoint=False, dtype=np.int16
    )
    return movie[:, :, indexes]

In [10]:
def uniform_sampling(movie, out_frame_num):
    movie_out = np.zeros(
        (movie.shape[0], movie.shape[1], out_frame_num, movie.shape[-1])
    )
    movie_frame_num = movie.shape[2]
    if movie_frame_num <= out_frame_num:
        movie_out[..., :movie_frame_num, :] = movie
        return movie_out
    for frame_idx, frame_group in enumerate(
        np.array_split(movie, out_frame_num, axis=2)
    ):
        frames = frame_group.shape[2]
        frame_taken = np.random.randint(0, frames)
        movie_out[..., frame_idx, :] = frame_group[..., frame_taken, :]
    return movie_out

In [11]:
def normalize(movie, origin_joints, coordinate_dim):
    # Change origin to joint val from first val frame
    origin = movie[..., 0, origin_joints].mean(axis=2)
    for frame_idx in range(1, movie.shape[2]):
        if (movie[..., frame_idx, origin_joints] != 0).all():
            origin = movie[..., frame_idx, origin_joints].mean(axis=2)
            break
    movie = (movie.reshape((-1, movie.shape[0], coordinate_dim)) - origin).reshape(
        movie.shape
    )

    for axis_idx in range(movie.shape[1]):
        joint_keypoints = movie[:, axis_idx, :, :]
        movie[:, axis_idx, :, :] = (
            joint_keypoints - joint_keypoints.mean()
        ) / joint_keypoints.std()
    return movie

In [12]:
def preprocess_data(
    dataset, out_frame_num, coordinate_dim, joint_number, sampling, origin_joints
):
    dataset_len = len(dataset["annotations"])
    labels, names = [], []
    skelets = torch.zeros(
        (dataset_len, 2, coordinate_dim, out_frame_num, joint_number),
        dtype=torch.float32,
    )
    for idx, anot in enumerate(tqdm(dataset["annotations"])):
        labels.append(anot["label"])
        names.append(anot["frame_dir"])
        movie = anot["keypoint"].astype(np.float32)

        movie = movie.transpose((0, 3, 1, 2))
        # Keypoint shape - (Person[2], Axis[3], Frame[x], Joint[25])
        handle_missing_joints(movie=movie.transpose((2, 0, 1, 3)))
        movie = (
            even_spaced_sampling(movie, out_frame_num)
            if sampling == "even_spaced"
            else uniform_sampling(movie, out_frame_num)
        )
        movie = normalize(movie, origin_joints, coordinate_dim)
        skelets[idx, : movie.shape[0], :, : movie.shape[2]] = torch.from_numpy(movie)
    labels = np.array(labels)
    names = np.array(names)
    return labels, names, skelets

In [13]:
def get_motion(raw_data):
    zero_frame = torch.zeros(
        (
            raw_data.shape[0],
            raw_data.shape[1],
            raw_data.shape[2],
            1,
            raw_data.shape[-1],
        )
    )

    motion = torch.diff(raw_data, axis=-2, prepend=zero_frame)
    return motion

In [14]:
def get_train_val_dataset(names, labels, dataset, raw_data, motion_data):
    is_train = np.isin(names, np.array(dataset["split"][EVAL_TYPE + "_train"]))

    is_val = np.isin(names, np.array(dataset["split"][EVAL_TYPE + "_val"]))

    train_dataset = torch.utils.data.TensorDataset(
        raw_data[is_train],
        motion_data[is_train],
        torch.from_numpy(labels[is_train]).type(torch.LongTensor),
    )

    val_dataset = torch.utils.data.TensorDataset(
        raw_data[is_val],
        motion_data[is_val],
        torch.from_numpy(labels[is_val]).type(torch.LongTensor),
    )
    return train_dataset, val_dataset

In [15]:
def get_train_val_loader(batch_size, train_dataset, val_dataset):
    g = torch.Generator()

    g.manual_seed(0)

    train_loader = data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        generator=g,
    )

    val_loader = data.DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        generator=g,
    )
    return train_loader, val_loader

### Build model

In [16]:
class Block(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        kernel=3,
        padding=1,
        groups=1,
        max_pool=False,
        dropout=0.1,
    ):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(
                in_channels, out_channels, kernel, padding=padding, groups=groups
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Dropout2d(dropout),
        )
        if max_pool:
            self.conv += nn.Sequential(
                nn.MaxPool2d(2),
            )

    def forward(self, x):
        x = self.conv(x)
        return x

In [17]:
class NetConv(nn.Module):
    def __init__(self, coordinate_dim, dropout=0.1):
        super().__init__()
        self.conv1 = nn.Sequential(
            Block(
                coordinate_dim,
                36,
                kernel=(coordinate_dim, 1),
                padding=0,
                groups=coordinate_dim,
                dropout=dropout,
            ),
            Block(36, 36, kernel=(coordinate_dim, 1), padding=0, dropout=dropout),
        )
        self.conv2 = nn.Sequential(
            Block(36, 64, kernel=3, padding=1, max_pool=True, dropout=dropout),
            Block(64, 128, kernel=3, padding=1, max_pool=True, dropout=dropout),
        )

    def forward(self, x):
        x0 = self.conv1(x[:, 0])
        x1 = self.conv1(x[:, 1])
        x0 = self.conv2(x0)
        x1 = self.conv2(x1)
        return x0, x1

In [18]:
class NetConv2(nn.Module):
    def __init__(self, dropout=0.1, input_channels=256):
        super().__init__()
        self.conv = nn.Sequential(
            Block(
                input_channels, 256, kernel=3, padding=1, max_pool=True, dropout=dropout
            ),
            Block(256, 512, kernel=3, padding=1, max_pool=True, dropout=dropout),
        )
        self.flatten = nn.Flatten()

    def forward(self, net_raw_1, net_raw_2, net_motion_1=None, net_motion_2=None):
        if torch.is_tensor(net_motion_1):
            x0 = torch.cat((net_raw_1, net_motion_1), dim=1)
            x1 = torch.cat((net_raw_2, net_motion_2), dim=1)
        else:
            x0 = net_raw_1
            x1 = net_raw_2
        x0 = self.conv(x0)
        x1 = self.conv(x1)
        x = torch.maximum(self.flatten(x0), self.flatten(x1))
        return x

In [19]:
class NetFF(nn.Module):
    def __init__(self, class_number, input_features=512, dropout=0.5):
        super().__init__()

        self.ff = nn.Sequential(
            # Linear 1
            nn.Linear(input_features, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(dropout),
            # Linear 2
            nn.Linear(256, class_number),
        )

    def forward(self, x):
        x = self.ff(x)
        return x

### Define train loop

In [20]:
def fileName(accuracy, net_name, skeleton_type, folder):
    return (
        folder
        + net_name
        + "_"
        + skeleton_type
        + "_"
        + f"{accuracy:.1f}".replace(".", "")
        + ".pt"
    )

In [21]:
from pathlib import Path
import os


def saveModels(accuracy, net_dict, skeleton_type, folder_str):
    Path(folder_str).mkdir(parents=True, exist_ok=True)

    for net_name, net in net_dict.items():
        torch.save(
            net.state_dict(), fileName(accuracy, net_name, skeleton_type, folder_str)
        )


def deleteModels(accuracy, net_dict, skeleton_type, folder_str):
    for net_name in net_dict.keys():
        file_name = fileName(accuracy, net_name, skeleton_type, folder_str)
        if os.path.isfile(file_name):
            os.remove(file_name)

### Train model

In [22]:
EPOCHS = 120
DATASET_NAME = "ntu60"  # (ntu60, ntu120)
EVAL_TYPE = "xsub"  # (xsub, xview, xset)
SCHEDULER = "ReduceLROnPlateau"  # (ReduceLROnPlateau, )

In [23]:
CLASS_NUMBER = 60 if DATASET_NAME == "ntu60" else 120
FOLDER_STR = "models/" + DATASET_NAME + "/" + EVAL_TYPE + "/"

In [24]:
sweep_config = {"method": "grid"}
metric = {"name": "val/val_max_accuracy", "goal": "maximize"}
sweep_config["metric"] = metric
parameters_dict = {
    "optimizer": {"value": "adam"},
    "eval_Type": {"value": EVAL_TYPE},
    "dataset": {"value": DATASET_NAME},
    "epochs": {"value": EPOCHS},
    "scheduler": {"value": SCHEDULER},
    "skeleton_type": {"value": "2D"},
    "batch_size": {"value": 128},
    "sampling": {"value": "even_spaced"},
    "out_frame_num": {"value": 32},
    "dropout_conv": {"value": 0.1},
    "dropout_ff": {"value": 0.5},
    "learning_rate": {"values": [1e-3, 5e-4, 2.5e-4, 1e-4, 5e-5]},
    "model": {"value": "J-CNN"},
}
sweep_config["parameters"] = parameters_dict

sweep_id = wandb.sweep(sweep_config, project="Inżynierka")

Create sweep with ID: cvou4ffa
Sweep URL: https://wandb.ai/kubuseg/In%C5%BCynierka/sweeps/cvou4ffa


In [25]:
def build_dataset(
    dataset_path,
    out_frame_num,
    coordinate_dim,
    joint_number,
    sampling,
    origin_joints,
    batch_size,
):
    random.seed(0)  # python random generator
    np.random.seed(0)  # numpy random generator
    torch.manual_seed(0)
    torch.cuda.manual_seed_all(0)

    dataset = get_input_dataset(dataset_path)
    labels, names, raw_data = preprocess_data(
        dataset, out_frame_num, coordinate_dim, joint_number, sampling, origin_joints
    )
    motion_data = get_motion(raw_data)
    train_dataset, val_dataset = get_train_val_dataset(
        names, labels, dataset, raw_data, motion_data
    )
    train_loader, val_loader = get_train_val_loader(
        batch_size, train_dataset, val_dataset
    )
    return train_loader, val_loader

In [26]:
def build_network(
    coordinate_dim,
    class_number,
    input_features=512,
    input_channels=256,
    dropout_conv=0.1,
    dropout_ff=0.5,
):
    net_raw = NetConv(coordinate_dim, dropout_conv).to(device)
    net_motion = NetConv(coordinate_dim, dropout_conv).to(device)
    net_conv_2 = NetConv2(dropout=dropout_conv, input_channels=input_channels).to(
        device
    )
    net_ff = NetFF(class_number, input_features, dropout_ff).to(device)
    return net_raw, net_motion, net_conv_2, net_ff

In [27]:
import torch.optim as optim


def build_crit_optim_sched(networks, learning_rate):
    net_raw, net_motion, net_conv_2, net_ff = networks
    params = (
        sum(p.numel() for p in net_raw.parameters() if p.requires_grad)
        + sum(p.numel() for p in net_motion.parameters() if p.requires_grad)
        + sum(p.numel() for p in net_ff.parameters() if p.requires_grad)
        + sum(p.numel() for p in net_conv_2.parameters() if p.requires_grad)
    )
    print(f"Model parameters: {params:,}")

    criterion = nn.CrossEntropyLoss()
    params = (
        list(net_raw.parameters())
        + list(net_motion.parameters())
        + list(net_conv_2.parameters())
        + list(net_ff.parameters())
    )
    optimizer = optim.Adam(params, lr=learning_rate)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="max", factor=0.5, patience=10, min_lr=1e-5, verbose=True
    )
    return criterion, optimizer, scheduler

In [28]:
def get_model_output(models, inputs, model_name):
    net_raw, net_motion, net_conv_2, net_ff = models
    raw, motion = inputs
    if model_name == "2S-CNN":
        raw_out_1, raw_out_2 = net_raw(raw)
        motion_out_1, motion_out_2 = net_motion(motion)
        out_conv = net_conv_2(raw_out_1, raw_out_2, motion_out_1, motion_out_2)
    elif model_name == "1S-CNN":
        raw_out_1, raw_out_2 = net_raw(torch.cat((raw, motion), dim=-1))
        out_conv = net_conv_2(raw_out_1, raw_out_2)
    elif model_name == "J-CNN":
        raw_out_1, raw_out_2 = net_raw(raw)
        out_conv = net_conv_2(raw_out_1, raw_out_2)
    outputs = net_ff(out_conv)
    return outputs


In [29]:
def getAccuarcy(loader, networks, model):
    net_raw, net_motion, net_conv_2, net_ff = networks

    correct, total = 0, 0
    net_raw.eval()

    net_motion.eval()
    net_conv_2.eval()

    net_ff.eval()

    with torch.no_grad():
        for raw, motion, labels in loader:
            raw, motion, labels = raw.to(device), motion.to(device), labels.to(device)

            outputs = get_model_output(
                models=(net_raw, net_motion, net_conv_2, net_ff),
                inputs=(raw, motion),
                model_name=model
            )

            # the class with the highest energy is what we choose as prediction

            _, predicted = torch.max(outputs.data, dim=1)

            total += labels.size(0)

            correct += (predicted == labels).sum().item()

    return 100 * correct / total


In [30]:
def train_epoch(
    networks, train_loader, val_loader, optimizer, criterion, scheduler, model
):
    net_raw, net_motion, net_conv_2, net_ff = networks
    net_raw.train()
    net_motion.train()
    net_conv_2.train()
    net_ff.train()
    running_loss = 0.0
    correct, total = 0, 0
    for i, (raw, motion, labels) in enumerate(train_loader):
        raw, motion, labels = raw.to(device), motion.to(
            device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        outputs = get_model_output(
            models=(net_raw, net_motion, net_conv_2, net_ff),
            inputs=(raw, motion),
            model_name=model
        )

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, dim=1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    training_acc = 100 * correct / total
    acc = getAccuarcy(val_loader, (net_raw, net_motion, net_conv_2, net_ff), model)
    scheduler.step(acc)
    return running_loss, training_acc, acc

In [31]:
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        POSE_MODEL, COORDINATE_DIM, JOINT_NUMBER, ORIGIN_JOINTS = (
            ("hrnet", 2, 17, [5, 6, 11, 12])
            if config.skeleton_type == "2D"
            else ("3danno", 3, 25, [1])
        )
        DATASET_PATH = DATASET_NAME + "_" + POSE_MODEL + ".pkl"

        train_loader, val_loader = build_dataset(
            DATASET_PATH,
            config.out_frame_num,
            COORDINATE_DIM,
            JOINT_NUMBER,
            config.sampling,
            ORIGIN_JOINTS,
            config.batch_size,
        )
        if config.model == "2S-CNN":
            input_channels, input_features = 256, 512
        elif config.model == "1S-CNN":
            input_channels, input_features = 128, 1024
        elif config.model == "J-CNN":
            input_channels, input_features = 128, 512

        net_raw, net_motion, net_conv_2, net_ff = build_network(
            COORDINATE_DIM,
            CLASS_NUMBER,
            input_features=input_features,
            input_channels=input_channels,
            dropout_conv=config.dropout_conv,
            dropout_ff=config.dropout_ff,
        )
        criterion, optimizer, scheduler = build_crit_optim_sched(
            (net_raw, net_motion, net_conv_2, net_ff), config.learning_rate
        )
        net_dict = {
            "net_raw": net_raw,
            "net_motion": net_motion,
            "net_conv_2": net_conv_2,
            "net_ff": net_ff,
        }

        best_acc = 0.0
        for epoch in tqdm(range(config.epochs)):
            training_loss, training_acc, val_acc = train_epoch(
                (net_raw, net_motion, net_conv_2, net_ff),
                train_loader,
                val_loader,
                optimizer,
                criterion,
                scheduler,
                model=config.model,
            )
            if val_acc > best_acc:
                deleteModels(best_acc, net_dict, config.skeleton_type, FOLDER_STR)
                best_acc = val_acc
                saveModels(best_acc, net_dict, config.skeleton_type, FOLDER_STR)
            print("[%d/%d] loss: %.1f" % (epoch + 1, config.epochs, training_loss))
            print("Training acc: %.3f %%" % training_acc)
            print("Validation acc: %.3f %%" % val_acc)
            metrics = {
                "train/train_loss": training_loss,
                "train/epoch": epoch + 1,
                "train/train_accuracy": training_acc,
            }
            val_metrics = {
                "val/val_accuracy": val_acc,
                "val/val_max_accuracy": best_acc,
            }
            wandb.log({**metrics, **val_metrics})
        print("Finished Training")
        print(f"Best acc: {best_acc:.3f}")

In [33]:
wandb.agent(sweep_id, train, count=12)
# wandb.agent(sweep_id="noad1a5y", project="Inżynierka", function=train, count=12)
wandb.finish()

wandb: Agent Starting Run: mqjwh8yg with config:
wandb: 	batch_size: 128
wandb: 	dataset: ntu60
wandb: 	dropout_conv: 0.1
wandb: 	dropout_ff: 0.5
wandb: 	epochs: 120
wandb: 	eval_Type: xsub
wandb: 	learning_rate: 0.001
wandb: 	model: J-CNN
wandb: 	optimizer: adam
wandb: 	out_frame_num: 32
wandb: 	sampling: even_spaced
wandb: 	scheduler: ReduceLROnPlateau
wandb: 	skeleton_type: 2D
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


  0%|          | 0/56578 [00:00<?, ?it/s]

Model parameters: 1,819,964


  0%|          | 0/120 [00:00<?, ?it/s]

[1/120] loss: 794.4
Training acc: 28.790 %
Validation acc: 38.982 %
[2/120] loss: 415.9
Training acc: 57.207 %
Validation acc: 61.412 %
[3/120] loss: 323.7
Training acc: 65.915 %
Validation acc: 69.030 %
[4/120] loss: 276.3
Training acc: 70.659 %
Validation acc: 69.503 %
[5/120] loss: 245.2
Training acc: 73.917 %
Validation acc: 70.589 %
[6/120] loss: 222.9
Training acc: 76.142 %
Validation acc: 72.512 %
[7/120] loss: 206.4
Training acc: 77.711 %
Validation acc: 75.781 %
[8/120] loss: 191.2
Training acc: 79.340 %
Validation acc: 75.065 %
[9/120] loss: 177.9
Training acc: 80.741 %
Validation acc: 73.652 %
[10/120] loss: 167.1
Training acc: 81.687 %
Validation acc: 76.794 %
[11/120] loss: 156.9
Training acc: 82.754 %
Validation acc: 77.637 %
[12/120] loss: 147.0
Training acc: 83.685 %
Validation acc: 77.776 %
[13/120] loss: 139.9
Training acc: 84.540 %
Validation acc: 77.413 %
[14/120] loss: 131.3
Training acc: 85.241 %
Validation acc: 75.914 %
[15/120] loss: 126.8
Training acc: 85.643 %

...

Model parameters: 1,819,964


  0%|          | 0/120 [00:00<?, ?it/s]

[1/120] loss: 1192.7
Training acc: 6.291 %
Validation acc: 11.191 %
[2/120] loss: 1071.1
Training acc: 12.280 %
Validation acc: 22.715 %
[3/120] loss: 943.9
Training acc: 20.369 %
Validation acc: 30.703 %
[4/120] loss: 816.0
Training acc: 29.261 %
Validation acc: 38.909 %
[5/120] loss: 712.7
Training acc: 36.764 %
Validation acc: 46.443 %
[6/120] loss: 625.8
Training acc: 44.214 %
Validation acc: 53.836 %
[7/120] loss: 561.0
Training acc: 49.153 %
Validation acc: 57.227 %
[8/120] loss: 509.0
Training acc: 53.157 %
Validation acc: 61.576 %
[9/120] loss: 463.5
Training acc: 56.906 %
Validation acc: 62.025 %
[10/120] loss: 425.7
Training acc: 59.806 %
Validation acc: 66.246 %
[11/120] loss: 394.5
Training acc: 62.680 %
Validation acc: 66.834 %
[12/120] loss: 367.9
Training acc: 64.835 %
Validation acc: 68.569 %
[13/120] loss: 345.8
Training acc: 66.357 %
Validation acc: 69.540 %
[14/120] loss: 327.3
Training acc: 68.020 %
Validation acc: 70.947 %
[15/120] loss: 309.9
Training acc: 69.746 

train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_accuracy,▁▃▄▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████████████████
train/train_loss,█▆▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/val_accuracy,▁▄▅▆▇▇▇▇▇▇▇█████████████████████████████
val/val_max_accuracy,▁▄▅▆▇▇▇▇▇▇██████████████████████████████
train/epoch,120
train/train_accuracy,96.07643
train/train_loss,36.68749
val/val_accuracy,83.98132
val/val_max_accuracy,84.31491


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
